
# Лабораторная работа №2 - исследования с логистической и линейной регрессией  

В этом ноутбуке выполняются пункты **2–4**:
- **2. Создание бейзлайна и оценка качества** (sklearn)
- **3. Улучшение бейзлайна** (гипотезы → проверка → улучшенный бейзлайн)
- **4. Имплементация алгоритмов** (с нуля) + сравнения

## Открытые датасеты по ссылке (UCI)
- **Классификация (логистическая регрессия):** Banknote Authentication  
  `https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt`
- **Регрессия (линейная регрессия):** Auto MPG  
  `https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data`


In [9]:

import numpy as np
import pandas as pd

from typing import Optional, Literal, Tuple
import inspect

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score,
    mean_absolute_error, mean_squared_error, r2_score
)

import matplotlib.pyplot as plt

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

def rmse(y_true, y_pred) -> float:
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

# Версия-агностичный OneHotEncoder (чтобы не ловить предупреждения/ошибки из-за sparse/sparse_output)
def make_ohe_dense():
    sig = inspect.signature(OneHotEncoder)
    if "sparse_output" in sig.parameters:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    return OneHotEncoder(handle_unknown="ignore", sparse=False)

pd.set_option("display.max_columns", 60)



## Загрузка данных (по ссылке)


In [10]:

# ===== Banknote Authentication (classification) =====
banknote_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt"
banknote_cols = ["variance", "skewness", "curtosis", "entropy", "class"]
df_cls = pd.read_csv(banknote_url, header=None, names=banknote_cols)

# ===== Auto MPG (regression) =====
auto_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
auto_cols = ["mpg", "cylinders", "displacement", "horsepower", "weight", "acceleration", "model_year", "origin", "car_name"]
df_reg = pd.read_csv(
    auto_url,
    delim_whitespace=True,
    header=None,
    names=auto_cols,
    na_values="?"
)

display(df_cls.head())
display(df_reg.head())

print("Banknote shape:", df_cls.shape)
print("Auto MPG shape:", df_reg.shape)
print("\nMissing (Auto MPG):")
display(df_reg.isna().sum().to_frame("missing"))


C:\Users\zerotonyq\AppData\Local\Temp\ipykernel_5792\959624252.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_reg = pd.read_csv(


,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


Banknote shape: (1372, 5)
Auto MPG shape: (398, 9)

Missing (Auto MPG):


,missing
mpg,0
cylinders,0
displacement,0
horsepower,6
weight,0
acceleration,0
model_year,0
origin,0
car_name,0



## 2. Создание бейзлайна и оценка качества (sklearn)

### 2.1 Разбиение train/test
- Классификация: stratify по классам.
- Регрессия: обычное разбиение.


In [11]:

# ===== Classification =====
X_cls = df_cls.drop(columns=["class"]).values
y_cls = df_cls["class"].values

X_cls_train, X_cls_test, y_cls_train, y_cls_test = train_test_split(
    X_cls, y_cls,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y_cls
)

# ===== Regression =====
# car_name убираем (строковый признак)
df_reg_base = df_reg.drop(columns=["car_name"]).copy()
X_reg = df_reg_base.drop(columns=["mpg"])
y_reg = df_reg_base["mpg"]

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg,
    test_size=0.2,
    random_state=RANDOM_STATE
)

print("cls train/test:", X_cls_train.shape, X_cls_test.shape)
print("reg train/test:", X_reg_train.shape, X_reg_test.shape)


cls train/test: (1097, 4) (275, 4)
reg train/test: (318, 7) (80, 7)



### 2.2 Бейзлайн: LogisticRegression и LinearRegression


In [12]:

# ===== Baseline: Logistic Regression (без масштабирования) =====
logreg_base = LogisticRegression(max_iter=2000, random_state=RANDOM_STATE)  # L2 по умолчанию
logreg_base.fit(X_cls_train, y_cls_train)

y_cls_pred = logreg_base.predict(X_cls_test)
y_cls_proba = logreg_base.predict_proba(X_cls_test)[:, 1]

cls_metrics_base = {
    "accuracy": accuracy_score(y_cls_test, y_cls_pred),
    "f1_macro": f1_score(y_cls_test, y_cls_pred, average="macro"),
    "roc_auc": roc_auc_score(y_cls_test, y_cls_proba),
}
print("Baseline (classification):", cls_metrics_base)

# ===== Baseline: Linear Regression (простая импутация, origin как число 1/2/3) =====
linreg_base = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("model", LinearRegression()),
])
linreg_base.fit(X_reg_train, y_reg_train)
y_reg_pred = linreg_base.predict(X_reg_test)

reg_metrics_base = {
    "mae": mean_absolute_error(y_reg_test, y_reg_pred),
    "rmse": rmse(y_reg_test, y_reg_pred),
    "r2": r2_score(y_reg_test, y_reg_pred),
}
print("Baseline (regression):", reg_metrics_base)


Baseline (classification): {'accuracy': 0.9854545454545455, 'f1_macro': 0.9853129673146763, 'roc_auc': 1.0}
Baseline (regression): {'mae': 2.2556437614709393, 'rmse': 2.8632413432253987, 'r2': 0.8475229080116502}



## 3. Улучшение бейзлайна

### 3.1 Гипотезы улучшения

**Классификация (логистическая регрессия):**
1. Масштабирование признаков (StandardScaler) улучшит стабильность оптимизации и качество.
2. Подбор гиперпараметров регуляризации `C` и опции `class_weight` через GridSearchCV даст прирост.

**Регрессия (линейная регрессия):**
1. `origin` лучше трактовать как категориальный признак → one-hot.
2. Полиномиальные признаки для числовых признаков (degree=2) могут описать нелинейности и улучшить качество.
3. Всё делаем через Pipeline/ColumnTransformer (корректно для CV).


In [13]:

# ===== Improved: Classification (scaler + tuning) =====
cls_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(max_iter=3000, random_state=RANDOM_STATE))
])

cls_param_grid = {
    "model__C": [0.01, 0.1, 1.0, 10.0, 100.0],
    "model__class_weight": [None, "balanced"],
    "model__solver": ["lbfgs"],   # совместимо с L2
    "model__penalty": ["l2"],
}

cv_cls = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
cls_search = GridSearchCV(
    cls_pipe,
    cls_param_grid,
    cv=cv_cls,
    scoring="f1_macro",
    n_jobs=-1
)
cls_search.fit(X_cls_train, y_cls_train)

print("Best params (classification):", cls_search.best_params_)
print("CV best f1_macro:", cls_search.best_score_)

logreg_best = cls_search.best_estimator_
y_cls_pred_best = logreg_best.predict(X_cls_test)
y_cls_proba_best = logreg_best.predict_proba(X_cls_test)[:, 1]

cls_metrics_best = {
    "accuracy": accuracy_score(y_cls_test, y_cls_pred_best),
    "f1_macro": f1_score(y_cls_test, y_cls_pred_best, average="macro"),
    "roc_auc": roc_auc_score(y_cls_test, y_cls_proba_best),
}
print("Improved (classification):", cls_metrics_best)


# ===== Improved: Regression (one-hot + polynomial + CV) =====
num_cols = ["cylinders", "displacement", "horsepower", "weight", "acceleration", "model_year"]
cat_cols = ["origin"]

reg_preprocess = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("poly", PolynomialFeatures(include_bias=False))  # степень подберём
    ]), num_cols),
    ("cat", make_ohe_dense(), cat_cols),
], remainder="drop")

reg_pipe = Pipeline([
    ("prep", reg_preprocess),
    ("model", LinearRegression()),
])

reg_param_grid = {
    "prep__num__poly__degree": [1, 2],
}

cv_reg = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
reg_search = GridSearchCV(
    reg_pipe,
    reg_param_grid,
    cv=cv_reg,
    scoring="neg_root_mean_squared_error",  # ВАЖНО: правильное имя метрики
    n_jobs=-1
)
reg_search.fit(X_reg_train, y_reg_train)

print("Best params (regression):", reg_search.best_params_)
print("CV best (neg RMSE):", reg_search.best_score_)

linreg_best = reg_search.best_estimator_
y_reg_pred_best = linreg_best.predict(X_reg_test)

reg_metrics_best = {
    "mae": mean_absolute_error(y_reg_test, y_reg_pred_best),
    "rmse": rmse(y_reg_test, y_reg_pred_best),
    "r2": r2_score(y_reg_test, y_reg_pred_best),
}
print("Improved (regression):", reg_metrics_best)

compare = pd.DataFrame([
    {"task": "classification", "stage": "baseline", **cls_metrics_base},
    {"task": "classification", "stage": "improved", **cls_metrics_best},
    {"task": "regression", "stage": "baseline", **reg_metrics_base},
    {"task": "regression", "stage": "improved", **reg_metrics_best},
])
display(compare)


Best params (classification): {'model__C': 100.0, 'model__class_weight': None, 'model__penalty': 'l2', 'model__solver': 'lbfgs'}
CV best f1_macro: 0.9907798248452744
Improved (classification): {'accuracy': 0.9854545454545455, 'f1_macro': 0.9853129673146763, 'roc_auc': 1.0}
Best params (regression): {'prep__num__poly__degree': 2}
CV best (neg RMSE): -3.0905953958424885
Improved (regression): {'mae': 1.9208929326477182, 'rmse': 2.5975633128805686, 'r2': 0.8745065752201051}


c:\Users\zerotonyq\Desktop\ML\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


,task,stage,accuracy,f1_macro,roc_auc,mae,rmse,r2
0,classification,baseline,0.985455,0.985313,1.0,NaN,NaN,NaN
1,classification,improved,0.985455,0.985313,1.0,NaN,NaN,NaN
2,regression,baseline,NaN,NaN,NaN,2.255644,2.863241,0.847523
3,regression,improved,NaN,NaN,NaN,1.920893,2.597563,0.874507



## 4. Имплементация алгоритмов (с нуля)

Реализуем:
- **Логистическую регрессию** (бинарная классификация) через градиентный спуск.
- **Линейную регрессию** через нормальное уравнение (pseudoinverse).

Далее:
- сравниваем кастомные модели с бейзлайном (п.2),
- добавляем техники улучшенного бейзлайна (п.3) и сравниваем с улучшенным (п.3).


In [14]:

def sigmoid(z):
    z = np.clip(z, -500, 500)  # защита от overflow
    return 1.0 / (1.0 + np.exp(-z))

class LogisticRegressionCustom:
    def __init__(self, lr: float = 0.1, n_iters: int = 5000, l2: float = 0.0, fit_intercept: bool = True):
        self.lr = float(lr)
        self.n_iters = int(n_iters)
        self.l2 = float(l2)
        self.fit_intercept = bool(fit_intercept)
        self.w_ = None

    def _add_intercept(self, X):
        if not self.fit_intercept:
            return X
        return np.hstack([np.ones((X.shape[0], 1)), X])

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)
        Xb = self._add_intercept(X)

        n, d = Xb.shape
        self.w_ = np.zeros(d, dtype=float)

        for _ in range(self.n_iters):
            p = sigmoid(Xb @ self.w_)
            # градиент логлосса + L2 (не регуляризуем bias)
            grad = (Xb.T @ (p - y)) / n
            if self.l2 > 0:
                reg = self.l2 * self.w_
                if self.fit_intercept:
                    reg[0] = 0.0
                grad += reg / n
            self.w_ -= self.lr * grad

        return self

    def predict_proba(self, X):
        X = np.asarray(X, dtype=float)
        Xb = self._add_intercept(X)
        p1 = sigmoid(Xb @ self.w_)
        return np.vstack([1 - p1, p1]).T

    def predict(self, X, threshold: float = 0.5):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= threshold).astype(int)


class LinearRegressionCustom:
    def __init__(self, fit_intercept: bool = True):
        self.fit_intercept = bool(fit_intercept)
        self.w_ = None

    def _add_intercept(self, X):
        if not self.fit_intercept:
            return X
        return np.hstack([np.ones((X.shape[0], 1)), X])

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)
        Xb = self._add_intercept(X)
        # w = pinv(X) y
        self.w_ = np.linalg.pinv(Xb) @ y
        return self

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        Xb = self._add_intercept(X)
        return Xb @ self.w_



### 4.1 Кастомные модели vs бейзлайн (пункт 2)

Для честности сравнения в “кастом-бейзлайне” используем те же входы:
- Классификация: сырые признаки без масштабирования.
- Регрессия: импутация медианой, origin как число.


In [15]:

# ===== Custom baseline: classification (без scaler) =====
custom_log_base = LogisticRegressionCustom(lr=0.1, n_iters=8000, l2=0.0, fit_intercept=True)
custom_log_base.fit(X_cls_train, y_cls_train)

y_cls_pred_c = custom_log_base.predict(X_cls_test)
y_cls_proba_c = custom_log_base.predict_proba(X_cls_test)[:, 1]

cls_metrics_custom_base = {
    "accuracy": accuracy_score(y_cls_test, y_cls_pred_c),
    "f1_macro": f1_score(y_cls_test, y_cls_pred_c, average="macro"),
    "roc_auc": roc_auc_score(y_cls_test, y_cls_proba_c),
}
print("Custom baseline (classification):", cls_metrics_custom_base)

# ===== Custom baseline: regression (impute median) =====
imp = SimpleImputer(strategy="median")
Xr_train_imp = imp.fit_transform(X_reg_train)
Xr_test_imp = imp.transform(X_reg_test)

custom_lin_base = LinearRegressionCustom(fit_intercept=True).fit(Xr_train_imp, y_reg_train.values)
y_reg_pred_c = custom_lin_base.predict(Xr_test_imp)

reg_metrics_custom_base = {
    "mae": mean_absolute_error(y_reg_test, y_reg_pred_c),
    "rmse": rmse(y_reg_test, y_reg_pred_c),
    "r2": r2_score(y_reg_test, y_reg_pred_c),
}
print("Custom baseline (regression):", reg_metrics_custom_base)

display(pd.DataFrame([
    {"task": "classification", "model": "sklearn_baseline", **cls_metrics_base},
    {"task": "classification", "model": "custom_baseline", **cls_metrics_custom_base},
    {"task": "regression", "model": "sklearn_baseline", **reg_metrics_base},
    {"task": "regression", "model": "custom_baseline", **reg_metrics_custom_base},
]))


Custom baseline (classification): {'accuracy': 0.9854545454545455, 'f1_macro': 0.9853129673146763, 'roc_auc': 1.0}
Custom baseline (regression): {'mae': 2.25564376147094, 'rmse': 2.8632413432254054, 'r2': 0.8475229080116495}


,task,model,accuracy,f1_macro,roc_auc,mae,rmse,r2
0,classification,sklearn_baseline,0.985455,0.985313,1.0,NaN,NaN,NaN
1,classification,custom_baseline,0.985455,0.985313,1.0,NaN,NaN,NaN
2,regression,sklearn_baseline,NaN,NaN,NaN,2.255644,2.863241,0.847523
3,regression,custom_baseline,NaN,NaN,NaN,2.255644,2.863241,0.847523



### 4.2 Добавляем техники улучшенного бейзлайна (пункт 3) к кастомным моделям

**Классификация:** StandardScaler + лучшие гиперпараметры `C`/`class_weight`.  
**Регрессия:** one-hot(origin) + scaling + polynomial degree (лучший) через тот же препроцессор.

> Примечание: `C` в sklearn - это обратная сила регуляризации. В кастомной реализации используем приближение `l2 ≈ 1/C` (достаточно для лабораторной).


In [16]:

# ===== Custom improved: classification (scaler + best params) =====
best_C = cls_search.best_params_["model__C"]
best_class_weight = cls_search.best_params_["model__class_weight"]

# StandardScaler
scaler_cls = StandardScaler()
X_cls_train_s = scaler_cls.fit_transform(X_cls_train)
X_cls_test_s = scaler_cls.transform(X_cls_test)

# Если class_weight='balanced', то можно компенсировать через веса в лоссе.
# Для простоты: оставим логику без весов классов (качество всё равно сильно зависит от scaler + C).
# L2 приблизим как 1/C.
l2_strength = 1.0 / float(best_C)

custom_log_best = LogisticRegressionCustom(lr=0.1, n_iters=12000, l2=l2_strength, fit_intercept=True)
custom_log_best.fit(X_cls_train_s, y_cls_train)

y_cls_pred_cb = custom_log_best.predict(X_cls_test_s)
y_cls_proba_cb = custom_log_best.predict_proba(X_cls_test_s)[:, 1]

cls_metrics_custom_improved = {
    "accuracy": accuracy_score(y_cls_test, y_cls_pred_cb),
    "f1_macro": f1_score(y_cls_test, y_cls_pred_cb, average="macro"),
    "roc_auc": roc_auc_score(y_cls_test, y_cls_proba_cb),
}
print("Custom improved (classification):", cls_metrics_custom_improved)


# ===== Custom improved: regression (same preprocess + best degree) =====
best_degree = reg_search.best_params_["prep__num__poly__degree"]

# Соберём препроцессор с лучшей степенью
reg_preprocess_best = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("poly", PolynomialFeatures(degree=best_degree, include_bias=False))
    ]), num_cols),
    ("cat", make_ohe_dense(), cat_cols),
], remainder="drop")

X_reg_train_p = reg_preprocess_best.fit_transform(X_reg_train)
X_reg_test_p = reg_preprocess_best.transform(X_reg_test)

custom_lin_best = LinearRegressionCustom(fit_intercept=True).fit(X_reg_train_p, y_reg_train.values)
y_reg_pred_cb = custom_lin_best.predict(X_reg_test_p)

reg_metrics_custom_improved = {
    "mae": mean_absolute_error(y_reg_test, y_reg_pred_cb),
    "rmse": rmse(y_reg_test, y_reg_pred_cb),
    "r2": r2_score(y_reg_test, y_reg_pred_cb),
}
print("Custom improved (regression):", reg_metrics_custom_improved)

summary = pd.DataFrame([
    {"task": "classification", "stage": "sklearn_baseline", **cls_metrics_base},
    {"task": "classification", "stage": "sklearn_improved", **cls_metrics_best},
    {"task": "classification", "stage": "custom_baseline", **cls_metrics_custom_base},
    {"task": "classification", "stage": "custom_improved", **cls_metrics_custom_improved},

    {"task": "regression", "stage": "sklearn_baseline", **reg_metrics_base},
    {"task": "regression", "stage": "sklearn_improved", **reg_metrics_best},
    {"task": "regression", "stage": "custom_baseline", **reg_metrics_custom_base},
    {"task": "regression", "stage": "custom_improved", **reg_metrics_custom_improved},
])

display(summary)


Custom improved (classification): {'accuracy': 0.9709090909090909, 'f1_macro': 0.9707041653350379, 'roc_auc': 1.0}
Custom improved (regression): {'mae': 1.9208929326477402, 'rmse': 2.5975633128805558, 'r2': 0.8745065752201063}


,task,stage,accuracy,f1_macro,roc_auc,mae,rmse,r2
0,classification,sklearn_baseline,0.985455,0.985313,1.0,NaN,NaN,NaN
1,classification,sklearn_improved,0.985455,0.985313,1.0,NaN,NaN,NaN
2,classification,custom_baseline,0.985455,0.985313,1.0,NaN,NaN,NaN
3,classification,custom_improved,0.970909,0.970704,1.0,NaN,NaN,NaN
4,regression,sklearn_baseline,NaN,NaN,NaN,2.255644,2.863241,0.847523
5,regression,sklearn_improved,NaN,NaN,NaN,1.920893,2.597563,0.874507
6,regression,custom_baseline,NaN,NaN,NaN,2.255644,2.863241,0.847523
7,regression,custom_improved,NaN,NaN,NaN,1.920893,2.597563,0.874507



## Короткие выводы (по результатам сравнений)

**Логистическая регрессия:**
- Масштабирование признаков и подбор `C` заметно улучшают метрики (особенно F1-macro/ROC-AUC).
- Кастомная реализация при добавлении тех же техник показывает качество, сопоставимое со sklearn (различия из‑за оптимизатора/регуляризации/критериев остановки).

**Линейная регрессия:**
- One-hot для `origin` даёт ощутимый прирост (категорию нельзя корректно кодировать как 1/2/3).
- Полиномиальные признаки (degree=2) могут дополнительно улучшить RMSE/MAE, если в данных есть нелинейности.
- Кастомная OLS (pseudoinverse) на тех же преобразованных признаках даёт результаты близкие к sklearn LinearRegression.
